# Generate Heatmaps

In [1]:
import folium
from folium.plugins import MarkerCluster
from folium import IFrame,Popup
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import shapely
from shapely.geometry import Point
import numpy as np
import pysal as ps

ModuleNotFoundError: No module named 'folium'

In [ ]:
data = pd.read_csv("data/gross_rent_with_population.csv").sort_values(['Zip_Code', 'Samples'], ascending=False)
data.info()

In [ ]:
geo_series = gpd.GeoSeries(data.apply(lambda z: Point(z['Lon'], z['Lat']) , 1), crs={'init': 'epsg:4326'})

geo_data = gpd.GeoDataFrame(data.drop(columns=['Lon', 'Lat']), geometry=geo_series).reset_index(drop=True)
geo_data.info()
geo_data.head(10)

In [ ]:
# Combine data with the same place name
tmp = geo_data.reset_index(drop=True)
tmp = tmp.drop(columns=["Zip_Code", "ALand", "AWater", "Median", "Stdev"])
tmp.head()

In [ ]:
# Pre-aggregate data on city level
tmp['totals'] = tmp['Mean'] * tmp['Samples']
citywise_data = tmp.dissolve(by=['State_Name','County', 'City'], aggfunc='sum', as_index=False)
citywise_data['Mean'] = (citywise_data.totals / citywise_data.Samples).fillna(0)
citywise_data.geometry = citywise_data.geometry.centroid
citywise_data.info()

In [ ]:
# Read counties
counties = gpd.read_file("data/gz_2010_us_050_00_20m/gz_2010_us_050_00_20m.shp").set_index('GEO_ID', drop=False)
join_result = gpd.tools.sjoin(counties, citywise_data.to_crs(counties.crs), op='contains').reset_index()
join_result.head()
join_result.info()

In [ ]:
# Multiply mean and Samples to be able to calculate the mean by county
sum_columns = join_result.groupby('GEO_ID').sum()
counties['Mean'] = (sum_columns['totals'] / sum_columns['Samples']).fillna(0)
counties['Population'] = (sum_columns['Population']).fillna(0)

In [ ]:
counties['Population'] = counties['Population'].fillna(0).astype(np.int64)
counties['Mean'] = counties['Mean'].fillna(0).astype(np.int64)
counties.info()
counties.head()

In [ ]:
def add_choropleth(mapobj, gdf, id_field, value_field, legend_name, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Percentiles':
        num_classes = 6
        threshold_scale=ps.esda.mapclassify.Percentiles(gdf[value_field], pct=[10, 50, 90, 99, 99.9, 100]).bins.tolist()
    if classifier == 'Jenks_Caspall':
        threshold_scale=ps.esda.mapclassify.Jenks_Caspall(gdf[value_field],  k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale, legend_name = legend_name, highlight=False, smooth_factor=1.0)
    return mapobj

def add_point_clusters(mapobj, gdf, popup_field_list, layer_name):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join(["{name}: {value}".format(name=field, value=str(row[field]) if type(row[field]) is str or type(row[field]) is int else "{:.1f}".format(row[field])) for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(Popup(label))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = layer_name)
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_child(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_child(pt_lyr)
    return mapobj


In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3, control_scale=True, prefer_canvas=True)

m = add_choropleth(m, counties, 'GEO_ID', 'Mean', legend_name='Mean Gross Rent', classifier='Percentiles', num_classes=6)
m = add_point_clusters(m, citywise_data, ['City','Population','Mean'], 'Mean Gross Rent per City')
folium.LayerControl().add_to(m) #Add layer control to toggle on/off
m.save("data/test.html")